importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# loading the dataset
credit_card_data = pd.read_csv('creditcard.csv')

In [ ]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [ ]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
39697,39927,-1.466679,2.425732,0.877724,3.950765,0.762477,1.162748,0.575625,-0.288354,-1.357990,2.907686,-0.890614,-0.462249,1.291626,-0.392713,1.434131,-0.347514,-0.124999,-0.270457,0.875010,0.129962,0.186051,0.571010,-0.107517,-0.754306,-0.752828,0.113179,-2.079421,-0.809173,1.50,0.0
39698,39927,-0.523165,-0.100021,0.892966,-1.900405,-0.156870,-0.783894,0.917683,-0.308345,-1.305284,0.514227,0.675557,-1.629259,-2.491738,0.386368,-0.595120,1.352518,-0.228732,-0.936432,0.660506,-0.071640,-0.082504,-0.414677,-0.063392,-0.087455,-0.303383,-0.682889,-0.178417,-0.137169,100.92,0.0
39699,39928,-2.768425,-1.007072,2.151127,0.117797,1.283178,1.869731,-0.562240,0.820374,0.348797,-0.122472,1.700394,1.375019,0.623614,-0.610102,0.577459,-0.941449,0.571209,-1.502828,-0.785017,-0.491556,-0.182963,0.778210,0.904077,-1.288631,0.212441,0.483975,-0.027614,-0.582813,11.99,0.0
39700,39928,1.201327,0.158614,-0.325263,0.471667,0.086446,-0.770357,0.422151,-0.205277,-0.451865,0.161808,1.063432,0.650581,-0.207706,0.742138,0.054512,0.176587,-0.598347,0.093661,0.320508,-0.023074,0.027664,-0.018485,-0.199382,0.053605,0.683829,0.428416,-0.077342,-0.006394,45.00,0.0
39701,39929,1.097669,-1.315782,0.659681,-0.683915,-1.342612,0.332629,-1.110900,0.194811,-0.248825,0.624948,0.324492,-0.279047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data information
credit_card_data.info()


# Replace inf/-inf with NaN
credit_card_data['Class'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN with a placeholder value (e.g. 0 or -1), then convert to int
credit_card_data['Class'] = credit_card_data['Class'].fillna(0).astype(int)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39702 entries, 0 to 39701
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    39702 non-null  int64  
 1   V1      39702 non-null  float64
 2   V2      39702 non-null  float64
 3   V3      39702 non-null  float64
 4   V4      39702 non-null  float64
 5   V5      39702 non-null  float64
 6   V6      39702 non-null  float64
 7   V7      39702 non-null  float64
 8   V8      39702 non-null  float64
 9   V9      39702 non-null  float64
 10  V10     39702 non-null  float64
 11  V11     39702 non-null  float64
 12  V12     39702 non-null  float64
 13  V13     39701 non-null  float64
 14  V14     39701 non-null  float64
 15  V15     39701 non-null  float64
 16  V16     39701 non-null  float64
 17  V17     39701 non-null  float64
 18  V18     39701 non-null  float64
 19  V19     39701 non-null  float64
 20  V20     39701 non-null  float64
 21  V21     39701 non-null  float64
 22

In [ ]:
# distribution of legit transection and fraud transection
credit_card_data['Class'].value_counts()

,count
Class,
0,39598
1,104


This is highly unbalanced dataset

0---> Normal Transection
1---> FRaud transection

In [ ]:
# seperating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [ ]:
print(legit.shape)
print(fraud.shape)

(39598, 31)
(104, 31)


In [ ]:
# statistical measure of the data

legit.Amount.describe()

,Amount
count,39597.000000
mean,87.420700
std,234.514383
min,0.000000
25%,7.420000
50%,23.500000
75%,79.000000
max,7879.420000


In [ ]:
fraud.Amount.describe()

,Amount
count,104.000000
mean,97.070769
std,255.012160
min,0.000000
25%,1.000000
50%,3.775000
75%,99.990000
max,1809.680000


In [ ]:
# compare the values for both transections
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,25515.089702,-0.194278,0.036933,0.744160,0.174523,-0.217442,0.107157,-0.093208,0.031787,0.222422,-0.054176,0.409489,-0.445753,0.247772,0.260637,0.097049,0.010676,0.171875,-0.082776,-0.034697,0.046047,-0.031386,-0.111706,-0.039470,0.007851,0.135895,0.022779,0.005987,0.003920,87.420700
1,20683.201923,-7.697309,5.766703,-10.853666,5.865850,-5.424996,-2.275717,-7.641867,3.827809,-2.924282,-6.655401,5.450235,-8.328284,0.398577,-8.293300,-0.018106,-4.914592,-7.959096,-2.830897,0.486825,0.663205,0.626795,-0.345972,-0.342944,-0.230514,0.299242,0.176522,0.811431,0.099914,97.070769


Undersampling

In [ ]:
# build a sample datset containing similar distribution of normal transection and fraud transection
legit_sample = legit.sample(n= 104)

In [ ]:
# concatenating two dataframe
new_dataset = pd.concat([legit_sample, fraud], axis = 0)

In [ ]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
23320,32697,1.323101,-1.393103,0.433789,-1.327880,-1.717583,-0.691175,-1.001486,-0.132899,-1.601772,1.389866,-0.889099,-1.398932,-0.218277,-0.213081,0.885398,-0.030764,0.345214,0.156854,-0.131988,-0.150140,-0.338124,-0.900733,0.052258,-0.037719,0.112664,-0.396570,0.017171,0.042853,118.75,0
11711,20012,-2.009341,1.905082,1.446557,-0.917460,0.394477,-0.318634,1.129613,-0.800929,2.111436,1.351459,2.126693,-2.395981,1.184244,0.716242,-0.871166,0.356207,-0.726267,0.299547,-0.578819,0.076667,-0.425917,-0.576998,-0.167463,-0.111614,-0.536490,-0.476895,-2.800340,-1.826252,8.92,0
5655,5901,-0.258057,0.969750,-0.214803,-0.108414,2.770432,3.497384,0.511870,0.701241,0.261665,-0.619741,0.766226,-2.744150,1.536758,2.061360,0.352330,-0.078137,0.131326,0.317814,0.887876,0.159638,-0.455998,-1.312780,0.096536,0.875702,-0.231350,-0.743328,0.105022,0.138387,45.55,0
29531,35514,-0.649549,-0.299157,1.972117,-1.178714,0.431279,1.920471,0.026065,0.602951,0.617086,-0.912256,1.689862,1.496870,0.027045,-0.638409,-0.762284,-1.073327,0.616985,-2.048931,-1.551093,-0.047932,0.080991,0.624248,0.290490,-0.956928,-1.272972,0.719053,0.008761,-0.059680,60.00,0
1543,1212,0.985049,-0.054746,1.254230,2.516709,-0.702219,0.580114,-0.629496,0.332124,-0.121873,0.719601,0.402916,0.184405,-0.739225,0.043623,-0.437277,1.124956,-0.894525,0.789202,-0.666122,-0.047151,0.164497,0.365784,-0.146898,0.005629,0.355157,0.105025,0.016544,0.030508,68.02,0


In [ ]:
new_dataset['Class'].value_counts()

,count
Class,
0,104
1,104


In [ ]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,27126.855769,-0.226302,-0.002008,0.640178,0.155526,-0.155233,0.094297,-0.120168,-0.045371,0.157816,-0.128739,0.383998,-0.452390,0.347322,0.015544,0.021649,-0.117310,-0.034675,-0.162656,0.003576,0.150412,0.039018,-0.160153,-0.102246,-0.032907,0.181684,-0.003124,-0.030765,0.002309,109.099423
1,20683.201923,-7.697309,5.766703,-10.853666,5.865850,-5.424996,-2.275717,-7.641867,3.827809,-2.924282,-6.655401,5.450235,-8.328284,0.398577,-8.293300,-0.018106,-4.914592,-7.959096,-2.830897,0.486825,0.663205,0.626795,-0.345972,-0.342944,-0.230514,0.299242,0.176522,0.811431,0.099914,97.070769


In [ ]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [ ]:
print(X)

        Time        V1        V2  ...       V27       V28  Amount
23320  32697  1.323101 -1.393103  ...  0.017171  0.042853  118.75
11711  20012 -2.009341  1.905082  ... -2.800340 -1.826252    8.92
5655    5901 -0.258057  0.969750  ...  0.105022  0.138387   45.55
29531  35514 -0.649549 -0.299157  ...  0.008761 -0.059680   60.00
1543    1212  0.985049 -0.054746  ...  0.016544  0.030508   68.02
...      ...       ...       ...  ...       ...       ...     ...
30473  35942 -4.194074  4.382897  ...  0.473934  0.473757   14.46
30496  35953 -4.844372  5.649439  ...  0.210214  0.391855  111.70
31002  36170 -5.685013  5.776516  ...  0.303445  0.219380  111.70
33276  37167 -7.923891 -5.198360  ...  1.937421 -1.552593   12.31
39183  39729 -0.964567 -1.643541  ... -0.308555 -0.164500  776.83

[208 rows x 30 columns]


In [ ]:
print(Y)

23320    0
11711    0
5655     0
29531    0
1543     0
        ..
30473    1
30496    1
31002    1
33276    1
39183    1
Name: Class, Length: 208, dtype: int64


Split the data into traina nd test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(208, 30) (166, 30) (42, 30)


In [ ]:
# model training

model = LogisticRegression()

In [ ]:
#training the logestic regression model with training data
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# evaluation

# accuracy on training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy on Training data :', training_data_accuracy)

Accuracy on Training data : 0.9819277108433735


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy on Test daat :', test_data_accuracy)

Accuracy on Test daat : 0.9523809523809523


In [ ]:
import pickle

# Save the trained logistic regression model
with open('logistic_model.pkl', 'wb') as file:
    pickle.dump(model, file)
